In [1]:
# stdlib
import os

# os.environ["ORCHESTRA_DEPLOYMENT_TYPE"] = "remote"
# os.environ["DEV_MODE"] = "True"
# os.environ["TEST_EXTERNAL_REGISTRY"] = "k3d-registry.localhost:5800"
# os.environ["CLUSTER_HTTP_PORT_LOW"] = "9083"

In [2]:
# stdlib

environment = os.environ.get("ORCHESTRA_DEPLOYMENT_TYPE", "python")
low_port = os.environ.get("CLUSTER_HTTP_PORT_LOW", "auto")
print(environment, low_port)

python auto


In [3]:
# syft absolute
import syft as sy
from syft import test_settings
import pandas as pd

# Only low side server and login as DS

In [4]:
server_low = sy.orchestra.launch(
    name="bigquery-low",
    server_side_type="low",
    dev_mode=True,
    n_consumers=1,
    create_producer=True,
    port=low_port,
)

Autoreload enabled
Starting bigquery-low server on 0.0.0.0:62820


INFO:     Will watch for changes in these directories: ['/Users/koen/workspace/PySyft/packages/syft/src/syft']
INFO:     Uvicorn running on http://0.0.0.0:62820 (Press CTRL+C to quit)
INFO:     Started reloader process [13539] using WatchFiles


Waiting for server to start

INFO:     Started server process [13541]
INFO:     Waiting for application startup.
INFO:     Application startup complete.


 Done.


SyftInfo: You have launched a development server at http://0.0.0.0:62820.It is intended only for local use.

In [5]:
ds_client = server_low.login(
    email="data_scientist@openmined.org", password="verysecurepassword"
)

Logged into <bigquery-low: Low side Datasite> as <data_scientist@openmined.org>


# Low side research

In [6]:
assert len(ds_client.custom_api.api_endpoints()) == 3

In [7]:
dataset_1 = test_settings.get("dataset_1", default="dataset_1")
dataset_2 = test_settings.get("dataset_2", default="dataset_2")
table_1 = test_settings.get("table_1", default="table_1")
table_2 = test_settings.get("table_2", default="table_2")
table_2_col_id = test_settings.get("table_2_col_id", default="table_id")
table_2_col_score = test_settings.get("table_2_col_score", default="colname")

# Test the schema

In [8]:
res = ds_client.api.services.bigquery.schema()
assert isinstance(res.get(), pd.DataFrame)

WARN: private key is based on server name: bigquery-low in dev_mode. Don't run this in production.
Document Store's SQLite DB path: /var/folders/q1/ryq93kwj055dlbpngxv1c7z40000gn/T/syft/4a471a09f56b4a1d809c0a7614074283/db/4a471a09f56b4a1d809c0a7614074283.sqlite
Action Store's SQLite DB path: /var/folders/q1/ryq93kwj055dlbpngxv1c7z40000gn/T/syft/4a471a09f56b4a1d809c0a7614074283/db/4a471a09f56b4a1d809c0a7614074283.sqlite
Adding producer for queue: api_call on: tcp://localhost:62829


# Test the mock

In [9]:
result = ds_client.api.services.bigquery.test_query.mock(
    sql_query=f"SELECT * from {dataset_2}.{table_2} limit 10"
)
assert len(result) == 10

In [10]:
with sy.raises(sy.SyftException, show=True):
    ds_client.api.services.bigquery.test_query.private(
        sql_query=f"SELECT * from {dataset_2}.{table_2} limit 10"
    )

with sy.raises successfully caught the following exception:


In [14]:
FUNC_NAME = "large_sample"
LARGE_SAMPLE_QUERY = f"SELECT * FROM {dataset_2}.{table_2} LIMIT 10000"

In [15]:
mock_res = ds_client.api.services.bigquery.test_query(sql_query=LARGE_SAMPLE_QUERY)

SyftSuccess: Syft function 'execute_query' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.


# Submit a query

In [13]:
submission = ds_client.api.services.bigquery.submit_query(
    func_name=FUNC_NAME, query=LARGE_SAMPLE_QUERY
)

In [25]:
# todo: this is very noisy, but it actually passes
with sy.raises(
    sy.SyftException(
        public_message="*Please wait for the admin to allow the execution of this code*"
    )
):
    ds_client.code.large_sample(blocking=False)

In [29]:
with sy.raises(
    sy.SyftException(public_message="*Your code is waiting for approval*")
):
    ds_client.code.large_sample()

AssertionError: Expected <class 'syft.types.errors.SyftException'> to be raised, but got <class 'requests.exceptions.ConnectionError'>.

In [28]:
# todo: add email server shutdown

In [27]:
if environment != "remote":
    server_low.land()

Stopping bigquery-low


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [13541]
INFO:     Stopping reloader process [13539]
